In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
from src.transformers.models.bert import BertModel

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNA_bert_6", trust_remote_code=True)
model = BertModel.from_pretrained("zhihan1996/DNA_bert_6", trust_remote_code=True)

/home/cameron/miniconda3/envs/cam_env_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/home/cameron/miniconda3/envs/cam_env_2/lib/python3.11/site-packages/torch/_utils.py:830: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(" ".join([dna[i:i+6] for i in range(0, len(dna), 1)]), return_tensors = 'pt')


In [4]:
inputs['input_ids']

tensor([[   2,  728, 2899, 3389, 1254,  907, 3616, 2164,  449, 1782, 3019, 3870,
         3177,  406, 1611, 2334, 1129,  406, 1611, 2336, 1137,  439, 1741, 2855,
         3214,  554, 2204,  612, 2434, 1530, 2009, 3926, 3403, 1312, 1137,  438,
         1739, 2846, 3177,  407, 1616, 2353, 1208,  723, 2877, 3302,  907, 3614,
         2155,  416, 1650, 2490, 1753, 2904, 3411,    1,    1,    1,    1,    1,
            3]])

In [5]:
dict(sorted(tokenizer.vocab.items(), key=lambda item: item[1]))

{'[PAD]': 0,
 '[UNK]': 1,
 '[CLS]': 2,
 '[SEP]': 3,
 '[MASK]': 4,
 'AAAAAA': 5,
 'AAAAAT': 6,
 'AAAAAC': 7,
 'AAAAAG': 8,
 'AAAATA': 9,
 'AAAATT': 10,
 'AAAATC': 11,
 'AAAATG': 12,
 'AAAACA': 13,
 'AAAACT': 14,
 'AAAACC': 15,
 'AAAACG': 16,
 'AAAAGA': 17,
 'AAAAGT': 18,
 'AAAAGC': 19,
 'AAAAGG': 20,
 'AAATAA': 21,
 'AAATAT': 22,
 'AAATAC': 23,
 'AAATAG': 24,
 'AAATTA': 25,
 'AAATTT': 26,
 'AAATTC': 27,
 'AAATTG': 28,
 'AAATCA': 29,
 'AAATCT': 30,
 'AAATCC': 31,
 'AAATCG': 32,
 'AAATGA': 33,
 'AAATGT': 34,
 'AAATGC': 35,
 'AAATGG': 36,
 'AAACAA': 37,
 'AAACAT': 38,
 'AAACAC': 39,
 'AAACAG': 40,
 'AAACTA': 41,
 'AAACTT': 42,
 'AAACTC': 43,
 'AAACTG': 44,
 'AAACCA': 45,
 'AAACCT': 46,
 'AAACCC': 47,
 'AAACCG': 48,
 'AAACGA': 49,
 'AAACGT': 50,
 'AAACGC': 51,
 'AAACGG': 52,
 'AAAGAA': 53,
 'AAAGAT': 54,
 'AAAGAC': 55,
 'AAAGAG': 56,
 'AAAGTA': 57,
 'AAAGTT': 58,
 'AAAGTC': 59,
 'AAAGTG': 60,
 'AAAGCA': 61,
 'AAAGCT': 62,
 'AAAGCC': 63,
 'AAAGCG': 64,
 'AAAGGA': 65,
 'AAAGGT': 66,
 'AAAGGC'

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
device

device(type='cuda')

In [8]:
model = model.to(device)

In [9]:
inputs['input_ids'].shape


torch.Size([1, 61])

In [10]:
out = model(**inputs.to(device), output_attentions=True, return_dict=True)

In [12]:
out.key.shape

torch.Size([1, 12, 61, 64])

In [68]:
model.embeddings.forward(inputs.to(device)["input_ids"], inputs.to(device)["token_type_ids"])

tensor([[[-0.0622, -0.0385,  0.1013,  ..., -0.0403, -0.0234, -0.0343],
         [ 0.1521,  0.0156,  0.0175,  ..., -0.1897,  0.0935,  0.0241],
         [ 0.0246,  0.0086,  0.0833,  ..., -0.0325,  0.0045,  0.0782],
         ...,
         [-0.0537,  0.1426,  0.0506,  ...,  0.0769,  0.0726,  0.1250],
         [ 0.1343, -0.1633, -0.0138,  ..., -0.4566, -0.1181, -0.0063],
         [-0.0328, -0.0395,  0.0512,  ..., -0.0384, -0.0244, -0.0343]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [70]:
hidden_states = model.encoder.forward(model.embeddings.forward(inputs.to(device)["input_ids"], inputs.to(device)["token_type_ids"]),inputs.to(device)["attention_mask"] ,output_all_encoded_layers =True) # [1, sequence_length, 768]

In [73]:
hidden_states[0].shape

torch.Size([17, 768])

In [38]:
hidden_states[1].shape

torch.Size([1, 768])

In [8]:


# embedding with mean pooling
embedding_mean = torch.mean(hidden_states[0], dim=0)
print(embedding_mean.shape) # expect to be 768

# embedding with max pooling
embedding_max = torch.max(hidden_states[0], dim=0)[0]
print(embedding_max.shape) # expect to be 768

torch.Size([768])
torch.Size([768])


In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.current_device()

0

In [10]:
hidden_states.shape

torch.Size([1, 17, 768])

In [14]:
len(tokenizer.vocab)

4096

In [13]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(4101, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
   